In [32]:
###################################################################################################################
###################################################################################################################
##
##                                  IMPORT DES PACKAGES, FONCTIONS, LIBRAIRIES
## 
###################################################################################################################
###################################################################################################################
# Import des packages, librairies et fonctions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from datetime import datetime, timedelta

###################################################################################################################
###################################################################################################################
##
##                                                   FONCTIONS                                                   
##
###################################################################################################################
###################################################################################################################
def f_get_median_value(df, column:str, date, climat, range_days=3):
###################################################################################################################
##
## Fonction : f_get_median_value
## Description : Fonction qui retourne la valeur médiane d'une colonne sur l'intervalle [j-3range_days; j+range_days] 
##
## Return : Moyenne calculée
## 
###################################################################################################################
    date_min = date - timedelta(days=range_days)
    date_max = date + timedelta(days=range_days)
    
    ans = df[ (df['Climat'] == climat) 
                & (df['Date'] >= date_min)
                & (df['Date'] <= date_max)][column].median()

    return ans

def f_get_mean_value(df, colname, date, climat, range_days = 3):
###################################################################################################################
##
## Fonction : f_get_mean_value
## Description : Fonction qui retourne la valeur moyenne d'une colonne sur l'intervalle [j-3range_days; j+range_days]
## Return : Moyenne calculée
## 
###################################################################################################################
    date_min = date - timedelta(days=range_days)
    date_max = date + timedelta(days=range_days)
    
    tmp = df.loc[(df["Date"] >= date_min) & (df["Date"] <= date_max) & (df["Climat"] == climat)]

    moyenne = np.round( tmp[colname].mean(), 1) 
    return moyenne

def f_add_info_location (df):
###################################################################################################################
##
## Fonction : f_climat
## Description : Fonction qui ajoute au dataframe des infos complémentaires sur la location : 
##  Admin_name      :  Etat auquel appartient la location
##  Climat          : Climat associé à la location : 
##                          0 : Aride , Désert
##                          1 : Subtropical
##                          2 : Tropical
##                          3 : Tempéré
## 
## Latitude         : Latitude du lieu
## Longitude        : longitude du lieu
## Return : Dataframe avec nouvelles colonnes
## 
###################################################################################################################

    # Import du fichiers des données complémentaires sur les villes
    df_villes = pd.read_csv("villes.csv", sep=",")

    # Ajout des colonnes Etat et Climat
    df_new = df.merge(df_villes, left_on='Location', right_on='Ville')
    df_new = df_new.drop(columns = ['Ville', 'Country', 'Iso2', 'Capital', 'Population', 'Population_proper', 'Admin_name'], axis = 1)

    dictionnaire = { 'Aride' : 0, 'Désert': 0 , 
                    'Subtropical' : 1, 
                    'Tropical' : 2, 
                    'Tempéré' : 3}
    
    df_new["Climat"] = df_new["Climat"].replace(dictionnaire)
    df_new = df_new.astype({'Climat' : int})

    df_new.head()

    return df_new

def f_drop_na (df):
 ###################################################################################################################
##
## Fonction : f_drop_na
## Description : Suppression des lignes contenant des NA 
##  
## Return : df
## 
###################################################################################################################
   # Suppression des lignes où RainTomorrow est NA
    df = df.dropna(subset = ['RainTomorrow'])
    #df_meteo = df_meteo.dropna(subset = ['RainToday'])
    return df

def f_create_df_mean(df):
###################################################################################################################
##
## Fonction : f_create_df_mean
## Description : Création d'un dataframe contenant les moyennes par climat 
##  
## Return : df_mean
## 
###################################################################################################################
    agg_dictionary = {'MinTemp': 'mean',
                    'MaxTemp': 'mean',
                    'Rainfall': 'mean',
                    'Humidity3pm': 'mean',
                    'Temp9am': 'mean',
                    'Temp3pm': 'mean'}

    df_mean = df.groupby(['Climat', 'Date']).agg(agg_dictionary).reset_index()

    return df_mean

def f_maj_na_mean (df):
###################################################################################################################
##
## Fonction : f_maj_na_mean (df, df_mean)
## Description : Mise à jour des valeurs manquantes par la valeur moyenne calculée
##  
## Return : df_mean
## 
###################################################################################################################
    columns_temp = ['MinTemp', 'MaxTemp', 'Temp9am', 'Temp3pm']
    for column in columns_temp:
        if debug : print(f"Filling column {column}")
        index_na = df[column].isna()
        if debug : print(f"There are {df[column].isna().sum()} missing values before filling")
        df.loc[index_na, column] = df[index_na].apply(lambda x: f_get_mean_value(df, column, x['Date'], x['Climat']), axis=1)
        if debug : print(f"There are {df[column].isna().sum()} after filling")
    return df_meteo


def f_maj_na_median(df):
###################################################################################################################
##
## Fonction : f_maj_na_median (df, df_mean)
## Description : Mise à jour des valeurs manquantes par la valeur mediane calculée
##  
## Return : df_mean
## 
###################################################################################################################
    #Setting column
    column = 'Rainfall'
    if debug : print(f"Filling column {column}")

    #Counting missing values
    index_na = df[column].isna()
    if debug : print(f"There are {df[column].isna().sum()} missing values before filling")

    #Filling missing values
    df.loc[index_na, column] = df[index_na].apply(lambda x: 0 if x['RainToday'] == 0 else f_get_median_value(df, column, x['Date'], x['Climat']), axis=1)
    if debug : print(f"There are {df[column].isna().sum()} after filling")
    return df

###################################################################################################################
###################################################################################################################
##  
##                                             PROGRAMME PRINCIPAL
##  
###################################################################################################################
###################################################################################################################
debug = True
# Import du fichier de données
df_meteo = pd.read_csv("weatherAUS.csv")

# Découpage de la date en 3 colonnes
df_meteo['Date'] = pd.to_datetime(df_meteo['Date'], yearfirst = True)
df_meteo["Year"] = df_meteo["Date"].dt.strftime("%Y")
df_meteo["Month"] = df_meteo["Date"].dt.strftime("%m")
df_meteo["Day"] = df_meteo["Date"].dt.strftime("%d")
df_meteo.drop(labels = "Date", axis = 1, inplace = True)


# Ajout des données complémentaires climat
df_meteo = f_add_info_location(df_meteo) 

# Suppression des colonnes
#   Evaportaion : nombre de NA très élevé
#   Sunshine    : Nombre de NA très élevé
list_col = ["Evaporation", "Sunshine"]
df_meteo.drop(labels=list_col, axis = 1, inplace = True)
                     
# Remplacement des Yes/No par 1/0
df_meteo["RainToday"] = df_meteo["RainToday"].replace({"Yes": 1, "No": 0})
df_meteo["RainTomorrow"] = df_meteo["RainTomorrow"].replace({"Yes": 1, "No": 0})

# Création du dataframe des moyennes sur 6 jours
df_mean = f_create_df_mean(df_meteo)

# Mise à jour des NA par la moyenne
df_meteo = f_maj_na_mean(df_meteo)

# Mise à jour des NA par la mediane
df_meteo = f_maj_na_median(df_meteo)


df_meteo.to_csv("dataset.csv")

df_meteo.head()



C:\Users\gfser\AppData\Local\Temp\ipykernel_33872\2658185968.py:89: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new["Climat"] = df_new["Climat"].replace(dictionnaire)
C:\Users\gfser\AppData\Local\Temp\ipykernel_33872\2658185968.py:200: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_meteo["RainToday"] = df_meteo["RainToday"].replace({"Yes": 1, "No": 0})
C:\Users\gfser\AppData\Local\Temp\ipykernel_33872\2658185968.py:201: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. T

Filling column MinTemp
There are 1505 missing values before filling
There are 0 after filling
Filling column MaxTemp
There are 1276 missing values before filling
There are 0 after filling
Filling column Temp9am
There are 1807 missing values before filling
There are 0 after filling
Filling column Temp3pm
There are 3652 missing values before filling
There are 0 after filling
Filling column Rainfall
There are 3319 missing values before filling
There are 0 after filling


,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Day,Climat,Latitude,Longitude
0,2008-12-01,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,...,16.9,21.8,0.0,0.0,2008,12,01,3,-36.0806,146.9158
1,2008-12-02,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,...,17.2,24.3,0.0,0.0,2008,12,02,3,-36.0806,146.9158
2,2008-12-03,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,...,21.0,23.2,0.0,0.0,2008,12,03,3,-36.0806,146.9158
3,2008-12-04,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,...,18.1,26.5,0.0,0.0,2008,12,04,3,-36.0806,146.9158
4,2008-12-05,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,...,17.8,29.7,0.0,0.0,2008,12,05,3,-36.0806,146.9158
